In [ ]:
from elasticsearch import Elasticsearch

#es = Elasticsearch("http://localhost:9200")
es = Elasticsearch(
    "http://localhost:9200",
    http_auth=("elastic", "pass12345")
)
es.info()

In [16]:
from os import path
import sys, os
from datetime import datetime
from json import dumps, loads
from time import sleep
from random import randint
import numpy as np
# ssh -o ServerAliveInterval=60 -L 9092:localhost:9092 tunnel@128.2.204.215 -NTf
from kafka import KafkaConsumer, KafkaProducer

# Update this for your demo otherwise you'll see my data :)
#topic = 'test_anindita'
topic = 'movielog9'
consumer1 = KafkaConsumer(
    topic,
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='latest',
    enable_auto_commit = True,
    value_deserializer = lambda m : m.decode('utf-8')
)
consumer2 = consumer1 = KafkaConsumer(
    topic,
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit = True,
)

In [17]:
count = 0
for message in consumer2:
    message = message.value.decode('utf-8')
    val = message.split(",")[2]
    if val.startswith("GET"):
        count += 1
        if count % 1000 == 0:
            print(message)
        os.system(f"echo {message} >> ratings_log_20221109.csv")

2022-10-13T23:51:13,558362,GET /data/m/the+incredibles+2004/109.mpg
2022-10-13T23:51:16,524674,GET /data/m/st.+vincent+2014/75.mpg
2022-10-13T23:51:18,224120,GET /data/m/trouble+the+water+2008/88.mpg
2022-10-13T23:51:21,212379,GET /data/m/ghost+1990/83.mpg
2022-10-13T23:51:24,96185,GET /data/m/my+sisters+keeper+2009/110.mpg
2022-10-13T23:51:26,83060,GET /data/m/the+strange+love+of+martha+ivers+1946/20.mpg


KeyboardInterrupt: 

In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('ratings_log_20221109.csv', header = None, names = ['date', 'user', 'request'], parse_dates = ['date'])

In [20]:

df['movie_id'] = df['request'].str[12 : -8]
df_updated = df[['date','user','movie_id']]
df_updated.head()
df_updated.drop_duplicates(inplace=True)

C:\Users\Anindita Sahu\AppData\Local\Temp\ipykernel_14752\3173177892.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_updated.drop_duplicates(inplace=True)


In [10]:
mappings = {
        "properties": {
            "date": {"type": "text", "analyzer": "standard"},
            "user": {"type": "text", "analyzer": "standard"},
            "movie_id": {"type": "text", "analyzer": "standard"}
    }
}
es.indices.create(index="movies_watchpattern")

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'movies_watchpattern'}

In [ ]:
mappings = {
        "properties": {
            "date": {"type": "text", "analyzer": "standard"},
            "user": {"type": "text", "analyzer": "standard"},
            "movie_recommended": {"type": "text", "analyzer": "standard"}
    }
}

In [ ]:
i = 100
for i,row in df_updated.iterrows():
    print(i+100)

In [25]:

for i,row in df_updated.iterrows():
    doc = {
        "date": row["date"],
        "user": row["user"],
        "movie_id": row["movie_id"]
    }
    i= i+1
            
    es.index(index="movies_watchpattern", id=i+100, body=doc)

In [26]:
es.indices.refresh(index="movies_watchpattern")
es.cat.count(index="movies_watchpattern", format="json")

[{'epoch': '1668028751', 'timestamp': '21:19:11', 'count': '6406'}]

In [55]:
resp = es.search(
    index="movies_watchpattern",
    body={
        "query": {
            "bool": {
                "must": {
                    "match_phrase": {
                        "user": "51569",
                    }
                }
            },
        },            
    }
)
resp

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [ ]:
df = pd.read_csv("ratings_log_20221107.csv", names=['date', 'user', 'request'])
#df.head()
for user in df["user"]:
    print(type(user))

In [56]:
import requests
from datetime import datetime as dt

j=0
for user in df["user"]:
    # api-endpoint
    URL = "http://17645-team09.isri.cmu.edu:8082/recommend/"
    
    # location given here
    # user = str(user)
    
    # defining a params dict for the parameters to be sent to the API
    #PARAMS = {user}

    URL=URL+str(user)
    
    # sending get request and saving the response as response object
    r = requests.get(url = URL)
    
    # extracting data in json format
    data = r.text.split(",")
    x = dt.now().isoformat()
    for i in range(len(data)):
        doc = {
            "date": x,
            "user": user,
            "movie_recommended": data[i] 
        }
        es.index(index="movies_recommended", id=i+j, body=doc)
    j= j+ 20

In [ ]:
es.indices.refresh(index="movies_recommended")
es.cat.count(index="movies_recommended", format="json")